# Dawid Przybyliński 298836
## Sieci Kohonena - lab2

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import math
from matplotlib import pyplot as plt
from PIL import Image

In [2]:
from sklearn.metrics import homogeneity_score, completeness_score
import random

In [3]:
class SOM:
    
    def __init__(self, M, N, net='rect', radius=1, data_dim=28*28, start=0, stop=255):
        self.M = M
        self.N = N
        self.net = net
        self.radius = radius
        self.neurons = np.random.uniform(start, stop, size=(N*M, data_dim))
        
    @staticmethod
    def net_distance(self, x,y):
        """
        odległość między neuronami x, y na siatce dyskretnej
        odległość jest liczona za pomocą tw. Pitagorasa
        """
        if self.net == 'hex':
            v = (math.floor(x/self.N)-math.floor(y/self.N)) * 0.5
            mod1 = x%(2*self.N)
            mod2 = y%(2*self.N)
            if (mod1 < self.N and mod2 < self.N) or (mod1 >= self.N and mod2 >= self.N):
                diff = mod1-mod2
            else:
                if mod1>mod2:
                    mod1 = mod1 - self.N - 0.5
                else:
                    mod2 = mod2 - self.N - 0.5
                diff = mod1-mod2
            h = diff * math.sqrt(3)
            return self.radius * math.sqrt(v**2 + h**2)
        if self.net == 'rect':
            h = x%(self.N)-y%(self.N)
            v = math.floor(x/self.N)-math.floor(y/self.N)
            return self.radius * math.sqrt(v**2 + h**2)
    
    @staticmethod
    def decay(t,lam):
        """
        funckja alfa wygaszająca uczenie
        """
        return np.exp(-t/lam)
    
    @staticmethod
    def gauss(x, t):
        """
        funkcja gaussowska dla sąsiedztwa
        """
        return np.exp(-(x*t)**2)
    
    @staticmethod
    def mexican_hat(x, t):
        """
        funkcja meksykański kapelusz dla sąsiedztwa
        """
        return (2-4*(x*t)**2) * t**2 * np.exp(-(x*t)**2)
    
    @staticmethod
    def mod_mexican_hat(x, t):
        """
        funkcja meksykański kapelusz dla sąsiedztwa
        """
        return (1 - 2 * x ** 2 * t ** 2) * 2 * t ** 2 * np.exp(-(x * t) ** 2)
    
    def train(self, data, epochs=100, neighborhood="gauss"):
        """
        trenowanie sieci
        """
        # wybór funkcji sąsiedztwa
        if neighborhood == "gauss":
            neighborhood_fun = self.gauss
        elif neighborhood == "mexican_hat":
            neighborhood_fun = self.mexican_hat
        elif neighborhood == "mod_mexican_hat":
            neighborhood_fun = self.mod_mexican_hat
        else:
            print("Unknown neighborhood function, assuming gauss")
            neighborhood_fun = self.gauss
        
        # trenowanie 
        for epoch in range(epochs):
            print("Epoch: {}/{}".format(epoch+1,epochs), end='\r')
            obs_permutation = np.random.permutation(len(data))
            for obs in data[obs_permutation]:
                # znajdowanie zwycięzkiego neuronu dla obserwacji
                winner = np.argmin([np.linalg.norm(obs-neuron) for neuron in self.neurons])
                # przesunięcie wszystkich neuronów
                for i, neuron in enumerate(self.neurons):
                    self.neurons[i] += neighborhood_fun(self.net_distance(self, winner, i), epoch)*self.decay(epoch, epochs)*(obs-neuron)
    
    def predict(self, data):
        """
        dla każdej obserwacji znajduje najbliższy neuron 
        """
        return [np.argmin([np.linalg.norm(neuron-obs) for neuron in self.neurons]) for obs in data]

## Dane MNIST

Dane zawierają jasności kolejnych pikseli tworzących obrazek przedstawiający cyfry napisane odręcznie. Ze względu na dużą liczbę obserwacji w zbiorze, ograniczamy zbiór uczący do pierwszego tysiąca obserwacji. Dla poszczególnych odległości między sąsiednimi neuronami w przestrzeni dyskretnej (parametr *r* - radius, width) policzone zostaną miary kompletności i homogeniczności.

In [4]:
digits = pd.read_csv('datasets/mio2/mnist_test.csv')
digits

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9998,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
digits_array = np.asarray(digits)

In [6]:
def draw_digit(data = digits_array, idx = 0):
    single_digit = data[idx,1:1000].reshape(28,28).astype(np.uint8)
    img = Image.fromarray(single_digit, 'L')
    return img

### pierwsze trzy obserwacje

In [7]:
draw_digit(idx=0)

In [8]:
draw_digit(idx=1)

In [9]:
draw_digit(idx=2)

### Trenowanie sieci i testowanie

In [10]:
x = digits_array[:,1:1000]
som = SOM(2,5, data_dim=28*28)
som.train(x)
preds = som.predict(x)

In [11]:
true = digits['label']

In [12]:
def clust_raport(true, preds, r = None):
    if r:
        print('Chosen radius: {}'.format(r))
    print('Completeness score: {}'.format(completeness_score(true,preds)))
    print('Homogeneity score: {}'.format(homogeneity_score(true,preds)))

In [13]:
clust_raport(true, preds, r=1)

Chosen radius: 1
Completeness score: 0.42425551771394965
Homogeneity score: 0.3918723102512911


Dla r równego 1 mamy wyniki na poziomie około 0.4. Dla porówniania sprawdźmy wyniki losowego przyporządkowania:

In [14]:
randoms = []
for i in range(len(true)):
    randoms.append(random.randint(0,9))

In [15]:
clust_raport(true, randoms)

Completeness score: 0.0016382533152046622
Homogeneity score: 0.00163909607505529


Biorąc pod uwagę, jak niskie są wyniki dla losowego podziału, możemy stwierdzić, że sieć radzi sobie nieźle. Przetestujmy jej działanie dla kilku innych wartości parametru r:

In [16]:
som = SOM(2,5, data_dim=28*28, radius=0.5)
som.train(x)
preds = som.predict(x)
clust_raport(true, preds, r=0.5)

Chosen radius: 0.5
Completeness score: 0.42710873029500634
Homogeneity score: 0.40929895779174164


In [17]:
som = SOM(2,5, data_dim=28*28, radius=0.1)
som.train(x)
preds = som.predict(x)
clust_raport(true, preds, r=0.1)

Chosen radius: 0.1
Completeness score: 0.38997549620492666
Homogeneity score: 0.3480042337906732


In [18]:
som = SOM(2,5, data_dim=28*28, radius=10)
som.train(x)
preds = som.predict(x)
clust_raport(true, preds, r=10)

Chosen radius: 10
Completeness score: 0.3684025766945206
Homogeneity score: 0.32892034709028095


In [19]:
som = SOM(2,5, data_dim=28*28, radius=20)
som.train(x)
preds = som.predict(x)
clust_raport(true, preds, r=20)

Chosen radius: 20
Completeness score: 0.4018883102654407
Homogeneity score: 0.3814277215867789


Dla różnych wartości parametru dostajemy podobne wartości miar, na poziomie około 0.4.

## Human Activity Recognition Using Smartphones Data Set

Dane pochodzą z odczytu żyroskopu zainstalowanego w smartphonie osób wykonujących którąś z sześciu czynności (chodzenie, wchodzenie do góry, schodzenie na dół, siedzenie, leżenie)

In [20]:
df2_x = pd.read_csv('datasets/mio2/UCI HAR Dataset/UCI HAR Dataset/train/X_train.txt', header=None, delimiter=r"\s+")

In [21]:
df2_x

,0,1,2,3,4,5,6,7,8,9,...,551,552,553,554,555,556,557,558,559,560
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.074323,-0.298676,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,0.158075,-0.595051,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,0.414503,-0.390748,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,0.404573,-0.117290,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,0.087753,-0.351471,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7347,0.299665,-0.057193,-0.181233,-0.195387,0.039905,0.077078,-0.282301,0.043616,0.060410,0.210795,...,-0.070157,-0.588433,-0.880324,-0.190437,0.829718,0.206972,-0.425619,-0.791883,0.238604,0.049819
7348,0.273853,-0.007749,-0.147468,-0.235309,0.004816,0.059280,-0.322552,-0.029456,0.080585,0.117440,...,0.165259,-0.390738,-0.680744,0.064907,0.875679,-0.879033,0.400219,-0.771840,0.252676,0.050053
7349,0.273387,-0.017011,-0.045022,-0.218218,-0.103822,0.274533,-0.304515,-0.098913,0.332584,0.043999,...,0.195034,0.025145,-0.304029,0.052806,-0.266724,0.864404,0.701169,-0.779133,0.249145,0.040811
7350,0.289654,-0.018843,-0.158281,-0.219139,-0.111412,0.268893,-0.310487,-0.068200,0.319473,0.101702,...,0.013865,0.063907,-0.344314,-0.101360,0.700740,0.936674,-0.589479,-0.785181,0.246432,0.025339


In [22]:
df2_y = pd.read_csv('datasets/mio2/UCI HAR Dataset/UCI HAR Dataset/train/y_train.txt', header=None, delimiter=r"\s+")

In [23]:
df2_y

,0
0,5
1,5
2,5
3,5
4,5
...,...
7347,2
7348,2
7349,2
7350,2


In [52]:
true = np.asarray(df2_y).flatten()

In [47]:
som = SOM(2,5, data_dim=561)
som.train(np.asarray(df2_x))

In [49]:
preds = som.predict(np.asarray(df2_x))

In [53]:
clust_raport(true, preds, r=1)

Chosen radius: 1
Completeness score: 0.5429281867738113
Homogeneity score: 0.5983318917439483


In [55]:
som = SOM(2,5, data_dim=561, radius=0.1)
som.train(np.asarray(df2_x))
preds = som.predict(np.asarray(df2_x))
clust_raport(true, preds, r=0.1)

Chosen radius: 0.1
Completeness score: 0.5399635066977004
Homogeneity score: 0.6100190460332272


In [56]:
randoms = []
for i in range(len(true)):
    randoms.append(random.randint(0,9))
clust_raport(true, randoms)

Completeness score: 0.001305287131214839
Homogeneity score: 0.0016842974742633744


Tutaj wyniki są lepsze niż w przypadku zbioru MNIST.